<a href="https://colab.research.google.com/github/iam-fern/Generate_data/blob/main/ex_sampling_c_100rep_n30_x6_sas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

import statistics as stats
import scipy.stats as ss
import scipy.stats as st
from scipy.stats import f_oneway
from scipy import stats
import scipy.stats as stats
import time
import statsmodels.api as sm

from statsmodels.formula.api import ols

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df = pd.read_csv('https://github.com/iam-fern/Research/raw/main/sampling_100rep_n30_x6_sas.csv', index_col=['rep', 'index'])
df

x1         x2  ...         y_hat         e_hat
rep  index                        ...                            
0.0  0.0    37.111866  32.662013  ... -2.683605e+07  7.543497e+09
     1.0    13.141221  22.760397  ... -6.728472e+07  6.349529e+07
     2.0    48.008464  13.812278  ... -3.908786e+08 -7.096000e+09
     3.0    40.816091  12.057417  ... -3.621157e+08  5.413935e+09
     4.0    32.356143  25.512632  ... -1.455301e+08  2.640781e+07
...               ...        ...  ...           ...           ...
99.0 25.0   30.402216  37.226437  ... -1.972108e+07  1.149920e+09
     26.0   46.893965  37.557994  ... -6.673807e+07  1.342356e+10
     27.0   34.273458  46.545282  ...  1.085122e+08  1.809913e+09
     28.0   40.376947  33.754068  ...  3.210249e+06  4.331872e+09
     29.0   41.383995  13.887993  ... -1.860974e+08  5.232162e+09

[3000 rows x 7 columns]

In [3]:
# ลบข้อมูลที่มี missing values
table = df.dropna()
table

x1         x2  ...         y_hat         e_hat
rep  index                        ...                            
0.0  0.0    37.111866  32.662013  ... -2.683605e+07  7.543497e+09
     1.0    13.141221  22.760397  ... -6.728472e+07  6.349529e+07
     2.0    48.008464  13.812278  ... -3.908786e+08 -7.096000e+09
     3.0    40.816091  12.057417  ... -3.621157e+08  5.413935e+09
     4.0    32.356143  25.512632  ... -1.455301e+08  2.640781e+07
...               ...        ...  ...           ...           ...
99.0 25.0   30.402216  37.226437  ... -1.972108e+07  1.149920e+09
     26.0   46.893965  37.557994  ... -6.673807e+07  1.342356e+10
     27.0   34.273458  46.545282  ...  1.085122e+08  1.809913e+09
     28.0   40.376947  33.754068  ...  3.210249e+06  4.331872e+09
     29.0   41.383995  13.887993  ... -1.860974e+08  5.232162e+09

[3000 rows x 7 columns]

In [4]:
table.reset_index(inplace=True)
table

,rep,index,x1,x2,x3,e,Y,y_hat,e_hat
0,0.0,0.0,37.111866,32.662013,21.056391,7.516661e+09,7.516661e+09,-2.683605e+07,7.543497e+09
1,0.0,1.0,13.141221,22.760397,44.332542,-3.789637e+06,-3.789433e+06,-6.728472e+07,6.349529e+07
2,0.0,2.0,48.008464,13.812278,47.708400,-7.486879e+09,-7.486879e+09,-3.908786e+08,-7.096000e+09
3,0.0,3.0,40.816091,12.057417,48.246911,5.051818e+09,5.051819e+09,-3.621157e+08,5.413935e+09
4,0.0,4.0,32.356143,25.512632,39.472574,-1.191227e+08,-1.191223e+08,-1.455301e+08,2.640781e+07
...,...,...,...,...,...,...,...,...,...
2995,99.0,25.0,30.402216,37.226437,37.718295,1.130198e+09,1.130199e+09,-1.972108e+07,1.149920e+09
2996,99.0,26.0,46.893965,37.557994,25.317107,1.335683e+10,1.335683e+10,-6.673807e+07,1.342356e+10
2997,99.0,27.0,34.273458,46.545282,22.665875,1.918424e+09,1.918425e+09,1.085122e+08,1.809913e+09
2998,99.0,28.0,40.376947,33.754068,12.122648,4.335082e+09,4.335082e+09,3.210249e+06,4.331872e+09


In [5]:
table.set_index(['rep', 'index'], inplace=True)

In [6]:
table

x1         x2  ...         y_hat         e_hat
rep  index                        ...                            
0.0  0.0    37.111866  32.662013  ... -2.683605e+07  7.543497e+09
     1.0    13.141221  22.760397  ... -6.728472e+07  6.349529e+07
     2.0    48.008464  13.812278  ... -3.908786e+08 -7.096000e+09
     3.0    40.816091  12.057417  ... -3.621157e+08  5.413935e+09
     4.0    32.356143  25.512632  ... -1.455301e+08  2.640781e+07
...               ...        ...  ...           ...           ...
99.0 25.0   30.402216  37.226437  ... -1.972108e+07  1.149920e+09
     26.0   46.893965  37.557994  ... -6.673807e+07  1.342356e+10
     27.0   34.273458  46.545282  ...  1.085122e+08  1.809913e+09
     28.0   40.376947  33.754068  ...  3.210249e+06  4.331872e+09
     29.0   41.383995  13.887993  ... -1.860974e+08  5.232162e+09

[3000 rows x 7 columns]

In [7]:
def get_result(run_replicate):
    new_table = table.loc[run_replicate]

    # show index
    new_table.reset_index(inplace=True)

    new_table.drop(['index'], axis=1, inplace=True)


    # แบ่งข้อมูลออกเป็น 2 กลุ่มด้วย k-mean

    #Set number of clusters
    x = new_table[["x1", "x2", "x3"]]
    k_means = KMeans(n_clusters=2)

    #Run the clustering algorithm
    model = k_means.fit(x)
    model

    #Generate cluster predictions and store in y_kmeans
    y_kmeans = k_means.predict(x)
    y_kmeans

    # kmean_clusters.astype(float)
    kmean_clusters = pd.DataFrame({"clusters" : y_kmeans})

    # เชื่อมข้อมูลทั้ง 2 DataFrame เข้าด้วยกันด้วยคำสั่ง pd.concat
    data_clusters = pd.concat([new_table, kmean_clusters], axis=1)

    data_sort_clusters = data_clusters.sort_values('clusters')

    # print(data_clusters.groupby('clusters').groups)

    data_sort_clusters.reset_index(inplace=True)

    # กลุ่มที่ 1
    data_cluster1 = data_sort_clusters[data_sort_clusters.clusters == 1]

    y = data_cluster1[['Y']]
    x = data_cluster1[['x1', 'x2', 'x3']]
    x = sm.add_constant(x)
    model1 = sm.OLS(y, x).fit()

    DF1 = model1.df_resid

    SSR1 = model1.ssr

    MSE1 = model1.mse_resid

    # กลุ่มที่ 2
    data_cluster0 = data_sort_clusters[data_sort_clusters.clusters == 0]

    y = data_cluster0[['Y']]
    x = data_cluster0[['x1', 'x2', 'x3']]
    x = sm.add_constant(x)
    model0 = sm.OLS(y, x).fit()

    DF0 = model0.df_resid

    SSR0 = model0.ssr

    MSE0 = model0.mse_resid

    df1 = DF1+1
    df0 = DF0+1

    # คำนวณค่าสถิติ
    GQ_test_clusters = (SSR1/df1) / (SSR0/df0)


    # คำนวณค่า p-value
    # sf(x, dfn, dfd, loc=0, scale=1) : Survival function (also defined as 1 - cdf, but sf is sometimes more accurate)
    two_tailed = 2.0*(1.0-(ss.f.cdf(GQ_test_clusters, df1, df0)))

    # pdf(x, dfn, dfd, loc=0, scale=1) : Probability density function
    p_greater = ss.f.cdf(GQ_test_clusters, df1, df0)
    
    return df0, df1, SSR0, SSR1, MSE0, MSE1, GQ_test_clusters, two_tailed, p_greater

# สร้าง list ของ replicates
list_replicate=list({i[0] for i in table.index})
list_replicate.sort()

# สร้าง list เพื่อเก็บค่า ทุกรอบที่ run
list_df0=[]
list_df1=[]
list_SSR0=[]
list_SSR1=[]
list_MSE0=[]
list_MSE1=[]
list_GQ_test_clusters=[]
list_result_test=[]
list_two_tailed=[]
list_p_greater=[]
list_result_test01=[]

# ใช้ for loop สำหรับการ run ทุก replicates
for run_replicate in list_replicate:
    df0, df1, SSR0, SSR1, MSE0, MSE1, GQ_test_clusters, two_tailed, p_greater=get_result(run_replicate)

    list_df0.append(df0)
    list_df1.append(df1)
    list_SSR0.append(SSR0)
    list_SSR1.append(SSR1)
    list_MSE0.append(MSE0)
    list_MSE1.append(MSE1)
    list_GQ_test_clusters.append(GQ_test_clusters)
    list_two_tailed.append(two_tailed)
    list_p_greater.append(p_greater)

In [8]:
# สร้างตารางผลลัพธ์
alpha = 0.05
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
two_tailed_new=pd.DataFrame({"Replicate":list_replicate, "df0":list_df0, "df1":list_df1, "SSR0":list_SSR0,
                     "SSR1":list_SSR1, "MSE0":list_MSE0, "MSE1":list_MSE1, 
                     "GQ_test_clusters":list_GQ_test_clusters, "two_tailed":list_two_tailed, "greater":list_p_greater})
two_tailed_new.set_index(['Replicate'], inplace=True)
two_tailed_new=two_tailed_new[(two_tailed_new["two_tailed"]<=1) & (two_tailed_new["greater"]<=1)]
two_tailed_new["Result_test"]=two_tailed_new["two_tailed"].apply(lambda p_value: 'Reject005=0 : Heteroscedasticity' 
                                                                  if p_value < alpha else 'Reject005=1 : Homoscedasticity')
#two_tailed_new_show=two_tailed_new.drop(columns="greater")
two_tailed_new.astype(str)

,df0,df1,SSR0,SSR1,MSE0,MSE1,GQ_test_clusters,two_tailed,greater,Result_test
Replicate,,,,,,,,,,
0.0,11.0,13.0,1.018151963799806e+20,3.0844693964366624e+20,1.018151963799806e+19,2.5703911636972188e+19,2.5634048117908383,0.12634357508133753,0.9368282124593312,Reject005=1 : Homoscedasticity
3.0,14.0,10.0,9.348450880127796e+17,2.2223146224604227e+20,7.191116061636766e+16,2.4692384694004695e+19,332.8081316721921,1.4654943925052066e-14,0.9999999999999927,Reject005=0 : Heteroscedasticity
6.0,14.0,10.0,3.803280105791133e+19,4.062404146608814e+20,2.925600081377795e+18,4.513782385120905e+19,14.953844174117942,1.629740286102077e-05,0.9999918512985695,Reject005=0 : Heteroscedasticity
7.0,14.0,10.0,1.6753985016887853e+19,7.761737567919973e+20,1.2887680782221425e+18,8.624152853244415e+19,64.85879379821998,1.151681638944524e-09,0.9999999994241592,Reject005=0 : Heteroscedasticity
8.0,11.0,13.0,1.6776316562739376e+17,5.8878083885631865e+20,1.6776316562739376e+16,4.906506990469322e+19,2969.657668754743,2.220446049250313e-16,0.9999999999999999,Reject005=0 : Heteroscedasticity
10.0,11.0,13.0,2.258727801494344e+20,5.289353520489441e+20,2.2587278014943437e+19,4.407794600407867e+19,1.9814724120668838,0.2633049761547901,0.868347511922605,Reject005=1 : Homoscedasticity
12.0,9.0,15.0,2.375433277543002e+19,5.139821862250865e+20,2.9692915969287526e+18,3.671301330179189e+19,12.982444703899674,0.0005139675185756865,0.9997430162407122,Reject005=0 : Heteroscedasticity
13.0,9.0,15.0,4401105092506590.5,2.8466837502840485e+20,550138136563323.8,2.0333455359171777e+19,38808.6676929965,2.220446049250313e-16,0.9999999999999999,Reject005=0 : Heteroscedasticity
14.0,10.0,14.0,7.561755346965371e+16,7.10316604041654e+20,8401950385517079.0,5.463973877243493e+19,6709.672286481861,2.220446049250313e-16,0.9999999999999999,Reject005=0 : Heteroscedasticity


In [9]:
# สร้างตารางผลลัพธ์
alpha = 0.05
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
two_tailed_new=pd.DataFrame({"Replicate":list_replicate, "df0":list_df0, "df1":list_df1, "SSR0":list_SSR0,
                     "SSR1":list_SSR1, "MSE0":list_MSE0, "MSE1":list_MSE1,
                     "GQ_test_clusters":list_GQ_test_clusters, "two_tailed":list_two_tailed, "greater":list_p_greater})
two_tailed_new.set_index(['Replicate'], inplace=True)

two_tailed_new["two_tailed_new"]=two_tailed_new[["two_tailed","greater"]].apply(lambda value: value[0] if value[0]<0.5 else 2*value[1], axis=1)
two_tailed_new["Result_test"]=two_tailed_new["two_tailed_new"].apply(lambda p_value: 'Reject005=0 : Heteroscedasticity' 
                                                                      if p_value < alpha else 'Reject005=1 : Homoscedasticity')


two_tailed_new_show=two_tailed_new.drop(columns=["two_tailed", "greater"])
two_tailed_new_show.astype(str)

,df0,df1,SSR0,SSR1,MSE0,MSE1,GQ_test_clusters,two_tailed_new,Result_test
Replicate,,,,,,,,,
0.0,11.0,13.0,1.018151963799806e+20,3.0844693964366624e+20,1.018151963799806e+19,2.5703911636972188e+19,2.5634048117908383,0.12634357508133753,Reject005=1 : Homoscedasticity
1.0,12.0,12.0,1.1144084374207581e+21,1.314351919614666e+19,1.0130985794734165e+20,1.1948653814678781e+18,0.011794166981153398,2.2045834664063335e-09,Reject005=0 : Heteroscedasticity
2.0,7.0,17.0,3.5563440883570835e+20,1.967598862845028e+19,5.927240147261806e+19,1.2297492892781425e+18,0.02278147858938238,2.333442487632449e-09,Reject005=0 : Heteroscedasticity
3.0,14.0,10.0,9.348450880127796e+17,2.2223146224604227e+20,7.191116061636766e+16,2.4692384694004695e+19,332.8081316721921,1.4654943925052066e-14,Reject005=0 : Heteroscedasticity
4.0,14.0,10.0,1.3334737780833015e+20,5.321574097838468e+19,1.025749060064078e+19,5.912860108709409e+18,0.558706429734417,0.35799639625894275,Reject005=1 : Homoscedasticity
5.0,7.0,17.0,3.693945152355402e+20,9.684737377497394e+17,6.156575253925669e+19,6.052960860935871e+16,0.0010795593527560226,2.238028358952058e-20,Reject005=0 : Heteroscedasticity
6.0,14.0,10.0,3.803280105791133e+19,4.062404146608814e+20,2.925600081377795e+18,4.513782385120905e+19,14.953844174117942,1.629740286102077e-05,Reject005=0 : Heteroscedasticity
7.0,14.0,10.0,1.6753985016887853e+19,7.761737567919973e+20,1.2887680782221425e+18,8.624152853244415e+19,64.85879379821998,1.151681638944524e-09,Reject005=0 : Heteroscedasticity
8.0,11.0,13.0,1.6776316562739376e+17,5.8878083885631865e+20,1.6776316562739376e+16,4.906506990469322e+19,2969.657668754743,2.220446049250313e-16,Reject005=0 : Heteroscedasticity


In [10]:
two_tailed_new_show['Result_test'].value_counts()

Reject005=0 : Heteroscedasticity    70
Reject005=1 : Homoscedasticity      30
Name: Result_test, dtype: int64